In [1]:
# 기본 데이터 처리
import pandas as pd
import os
from dotenv import load_dotenv
import warnings

# LangChain 관련
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from tqdm.notebook import tqdm

# 커스텀 모듈
from prompt.prompts import PROMPT_DATA_SUMMARY, PROMPT_ERROR_FIX
from sample_func.funcs import FUNC_DATA_SUMMARY
from utils.vector_handler import load_vectorstore

# 에러 처리
import traceback

# ✅ 환경 변수 로드 및 설정
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')
load_dotenv()

print("🔄 환경 변수 로드 완료.")

# ✅ OpenAI API Key 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0,)
vectorstore = load_vectorstore()

# ✅ 결과 저장을 위한 객체 선언
results = {}
list_df = {}

# ✅ 데이터 로드: ../data 경로의 모든 pkl 파일 읽기
data_path = "../data"
for file in os.listdir(data_path):
    if file.endswith('.pkl'):
        file_path = os.path.join(data_path, file)
        df_name = file.replace('.pkl', '')
        list_df[df_name] = pd.read_pickle(file_path)
        print(f"✅ 데이터 로드 완료: {df_name} ({len(list_df[df_name])} rows)")

# ✅ globals()에 데이터프레임 등록 (해결책 1)
for name, df in list_df.items():
    globals()[name] = df  # ✅ `analyze_multiple_dataframes`에서 접근 가능하도록 등록

list_df_text = ", ".join(list_df.keys())
print(f"📊 분석 대상 데이터프레임: {list_df_text}")

# ✅ Stage 1 - LLM에 코드 요청
print("🛠️ LLM에 데이터 구조 분석 코드 요청 중...")
chain = ChatPromptTemplate.from_messages([
    ("system", PROMPT_DATA_SUMMARY),
    ("user", "### 참고 코드:\n{func_code}\n\n"),
    ("user", "### list_df:\n{list_df_text}\n\n")
]) | llm

response = chain.invoke({"func_code": FUNC_DATA_SUMMARY, "list_df_text": list_df_text}).content

attempt_count = 0  # 실행 시도 횟수
success = False  # 코드 실행 성공 여부

while attempt_count < 2:  # 최초 실행 + 1회 재시도 가능
    try:
        if "```python" in response:
            modified_code = response.split("```python")[-1].split("```")[0]
            print(f"[LOG] 실행할 코드:\n{modified_code}")
            exec(modified_code, globals())  # ✅ LLM이 생성한 코드 실행
            print(f"✅ [Stage 1] 데이터 구조 파악 완료!")
            success = True
            break  # 실행 성공 시 루프 종료
    except Exception as e:
        error_trace = traceback.format_exc()  # ✅ 에러 상세 정보 수집
        print(f"❌ {attempt_count+1}차 시도 : LLM 생성 코드 실행 중 오류 발생\n{error_trace}")

        if attempt_count == 0:  # 최초 실행 실패 시 1회만 재생성
            print("🔄 오류 메시지를 기반으로 코드 수정 요청 중...")

            # ✅ LLM에 더 명확한 오류 메시지 전달 (해결책 2)
            chain_error_fix = ChatPromptTemplate.from_messages([
                ("system", PROMPT_ERROR_FIX.format(error_trace=error_trace)),
                ("user", "### 기존 코드:\n{modified_code}\n\n")
            ]) | llm

            response = chain_error_fix.invoke({"modified_code": modified_code}).content
        else:
            print("❌ 코드 실행 최종적으로 실패. 프로세스를 중단합니다.")

        attempt_count += 1  # 재시도 횟수 증가


# ✅ Stage 1 결과 파일 로드 (엑셀)
stage1_path = "../output/stage1/eda_summary.xlsx"
print(f"📂 Stage 1 결과 파일 확인: {stage1_path}")

if os.path.exists(stage1_path):
    # 모든 시트를 딕셔너리 형태로 로드
    stage1_results = pd.read_excel(stage1_path, sheet_name=None)  # sheet_name=None으로 모든 시트 로드
    results["stage1"] = stage1_results  # 모든 시트의 결과를 저장
    print(f"✅ [Stage 1] 엑셀 파일 로드 완료: {stage1_path}")
    print(f"📊 로드된 시트: {', '.join(stage1_results.keys())}")
else:
    print("❌ Stage 1 결과 파일이 존재하지 않습니다.")
    raise FileNotFoundError("Stage 1 결과 파일을 찾을 수 없습니다.")

print("🔍 RAG 기반 컬럼 설명 검색 시작...")

from tqdm.notebook import tqdm  # Jupyter notebook용 tqdm

def search_column_descriptions(stage1_results_dict, vectorstore):
    """RAG를 사용하여 컬럼 설명을 검색하는 함수"""
    data = []
    
    # 전체 처리해야 할 행 수 계산
    total_rows = sum(len(df) for df in stage1_results_dict.values())
    
    with tqdm(total=total_rows, desc="🔍 컬럼 설명 검색 중") as pbar:
        for df_name, df in stage1_results_dict.items():
            df_after_llm = df[['데이터프레임명', '컬럼명']]
            
            for i, row in df_after_llm.iterrows():
                table_name = row['데이터프레임명']
                col = row['컬럼명']
                
                search_query = f"테이블명 : {table_name} | 컬럼명 : {col}"
                docs = vectorstore.similarity_search(search_query, k=1)
                
                if docs:
                    best_match = docs[0].page_content
                    data.append({
                        '데이터프레임명': table_name,
                        '컬럼명': col,
                        '컬럼설명': best_match.split('\n')[3].split('설명')[1].strip()
                    })
                else:
                    data.append({
                        '데이터프레임명': table_name,
                        '컬럼명': col,
                        '컬럼설명': "설명 없음"
                    })
                pbar.update(1)  # 진행바 업데이트
    
    return pd.DataFrame(data)

# ✅ Stage 2: RAG 기반 컬럼 설명 추가
rag_results = search_column_descriptions(stage1_results, vectorstore)
print("✅ [Stage 2] RAG 검색 완료!")

# ✅ Stage 1 결과와 RAG 결과를 통합하여 각 시트별로 저장
output_path = "../output/stage1/processed_data_info.xlsx"
try:
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name, df in stage1_results.items():
            # 각 시트별로 RAG 결과와 통합
            final_df = pd.merge(
                df,  # Stage 1 결과 (데이터 구조 정보)
                rag_results,  # RAG 검색 결과 (컬럼 설명)
                on=['데이터프레임명', '컬럼명'],
                how='left'
            )
            # 통합된 결과를 시트별로 저장
            final_df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"✅ {sheet_name} 시트 저장 완료")
    
    print(f"✅ [Stage 2] 최종 결과 엑셀 저장 완료: {output_path}")
    results["stage2"] = stage1_results  # 결과 저장
except Exception as e:
    print(f"❌ 결과 저장 중 오류 발생: {e}")

print("🎉 모든 단계가 성공적으로 완료되었습니다!")


🔄 환경 변수 로드 완료.
✅ 데이터 로드 완료: cust_enroll_history (28182 rows)
✅ 데이터 로드 완료: cust_intg (120000 rows)
✅ 데이터 로드 완료: product_info (30 rows)
📊 분석 대상 데이터프레임: cust_enroll_history, cust_intg, product_info
🛠️ LLM에 데이터 구조 분석 코드 요청 중...
[LOG] 실행할 코드:

import pandas as pd
import numpy as np

def summarize_data(df, df_name):
    # 주어진 데이터프레임(df)에 대한 EDA(탐색적 데이터 분석) 결과를 정리하여 반환하는 함수.
    # "컬럼 개요"에 범주형 변수 분포 정보 포함
    # 연속형 변수의 인스턴스(예제) 제외 (토큰 수 절감 목적)

    # ✅ 1. 이상치 탐지 (IQR 방식)
    outliers_info = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()

        if outlier_count > 0:
            outliers_info[col] = int(outlier_count)

    # ✅ 2. 컬럼 개요 (기본 통계 및 결측 정보 + 인스턴스 예제 + 범주형 분포 추가)
    columns_info = []
    for col in df.columns:
        

🔍 컬럼 설명 검색 중:   0%|          | 0/133 [00:00<?, ?it/s]

✅ [Stage 2] RAG 검색 완료!
✅ cust_enroll_history 시트 저장 완료
✅ cust_intg 시트 저장 완료
✅ product_info 시트 저장 완료
✅ [Stage 2] 최종 결과 엑셀 저장 완료: ../output/stage1/processed_data_info.xlsx
🎉 모든 단계가 성공적으로 완료되었습니다!
